# ANÁLISIS EXPLORATORIO DE DATOS

## 1. Importar Librerias

In [1]:
# Librerias

import pandas as pd
import os
from pathlib import Path

## 2. Traer Datos

### 2.1. Traer Datos de Defunciones de 1979 - 1991

In [2]:
# Ruta de los archivos
ruta_muertes = Path("data/raw/Muertes")

# Verificar si la carpeta existe
if not ruta_muertes.exists():
    print(f"❌ La carpeta {ruta_muertes} no existe")
else:
    # Listar todos los archivos .txt en la carpeta
    archivos = list(ruta_muertes.glob("Defun*.txt"))
    print(f"📁 Archivos encontrados: {[arch.name for arch in archivos]}")
    
    # Filtrar por años 1979-1991
    archivos_filtrados = [arch for arch in archivos if 1979 <= int(arch.stem[-4:]) <= 1991]
    
    dataframes = {}
    
    for archivo in archivos_filtrados:
        try:
            # Extraer año del nombre del archivo
            año = archivo.stem[-4:]
            
            # Leer archivo (prueba diferentes encodings si es necesario)
            df = pd.read_csv(
                archivo,
                sep='\t',        # ajusta el separador
                encoding='utf-8', # prueba 'latin-1', 'ISO-8859-1' si falla
                low_memory=False
            )
            
            dataframes[f'defun_{año}'] = df
            print(f"✅ {archivo.name}: {len(df)} filas, {len(df.columns)} columnas")
            
        except Exception as e:
            print(f"❌ Error con {archivo.name}: {e}")

📁 Archivos encontrados: ['Defun1979.txt', 'Defun1980.txt', 'Defun1981.txt', 'Defun1982.txt', 'Defun1983.txt', 'Defun1984.txt', 'Defun1985.txt', 'Defun1986.txt', 'Defun1987.txt', 'Defun1988.txt', 'Defun1989.txt', 'Defun1990.txt', 'Defun1991.txt', 'Defun1992.txt', 'Defun1993.txt', 'Defun1994.txt', 'Defun1995.txt', 'Defun1996.txt', 'Defun1997.txt', 'Defun1998.txt', 'Defun1999.txt', 'Defun2000.txt', 'Defun2001.txt', 'Defun2002.txt', 'Defun2003.txt', 'Defun2004.txt', 'Defun2005.txt', 'Defun2006.txt', 'Defun2007.txt', 'Defun2012.txt', 'Defun2013.txt']
✅ Defun1979.txt: 110400 filas, 14 columnas
✅ Defun1980.txt: 125573 filas, 14 columnas
✅ Defun1981.txt: 139505 filas, 14 columnas
✅ Defun1982.txt: 137678 filas, 14 columnas
✅ Defun1983.txt: 140292 filas, 14 columnas
✅ Defun1984.txt: 137189 filas, 14 columnas
✅ Defun1985.txt: 153947 filas, 14 columnas
✅ Defun1986.txt: 146346 filas, 14 columnas
✅ Defun1987.txt: 151957 filas, 14 columnas
✅ Defun1988.txt: 153067 filas, 14 columnas
✅ Defun1989.txt: 1